In [1]:
#import
from bs4 import BeautifulSoup
import requests
import numpy as np
import json
import pickle

In [2]:
valid = "../url_scraping/charter_URLs_2016.csv"
file = "../url_scraping/json.txt"

### Helper Function

In [3]:
def write_file(str, file_name):
    """This function writes a string str to a given file name"""
    with open(file_name, "a") as text_file:
        text_file.write(str)
        
def read_txt(txt_file):
    """This function returns a list of all the lines of a file and also the count
    of how many lines"""
    links = []
    count = 0
    with open(txt_file) as f:
        for line in f:   
            count +=1
            links += [line.rstrip()]
    return links, count

def readtxt(txt_file):
    """This function returns a list of tuples and the count of lines
    each element of a tuple is separeted by tabtab
    example abc 123
            cdw 234
            return [(abc, 123), (cdw, 234)], 2"""
    links = []
    count = 0
    with open(txt_file) as f:
        for line in f:   
            elems = line.split(',')
            #elem is a tuple (name, link)
            elem =  (elems[0].rstrip(), elems[1].rstrip())
            count +=1
            links += [elem]
    return links, count

In [4]:
links, count = readtxt(valid)
print(links[0][1])

CHARTAUTHN2


In [5]:
url = 'https://www.polk.edu/lakeland-gateway-to-college-high-school/'
# credits: parsing team
def parse_data(url):
    
    data  = requests.get(url).text
    soup = BeautifulSoup(data, "lxml")
    hrefs = soup('a')
    paragraphs = soup('p')
    #only want unique hrefs
    links = []
    for each in hrefs:
        links.append(each.get('href'))
    links = set(links)
    return links, paragraphs
#TESTING
links, paragraphs = parse_data(url)

In [6]:
urlkey = ['about', 'department', 'home', 'community', 'life', 'curriculum', 'hist', 'phil', 'academics', 'moral', 'focus', 'beliefs', 'program', 'resources', 'mission', 'value', 'core']
urlkey = [s.upper() for s in urlkey]
#now parse through all of those links based on the key word search,
#and get a new list which contains the url's we actually want
# credits parsing team
def get_final_list(links):
    final_list = []
    for k in links:
        try:
            j = k.upper()
            for i in range(len(urlkey)):
                if isinstance(j, list) and len(j) == 1:
                    j = j[0].split(urlkey[i])
                elif isinstance(j, list) and len(j) > 1:
                    if requests.get(url + k):
                        final_list.append(k)
                        break
                else:
                    j = j.split(urlkey[i])
        except:
            pass

    final_list += ['/home']
    for i in range(len(final_list)):
        each = final_list[i].upper()
        # split with these keywords because we those pages with tag will not likely to give us 
        # the info we want
        k, m, n, o = each.split('FREQ'), each.split('ADMI'), each.split('TRUSTE'), each.split('ALUM')
        if len(k) > 1 or len(m) > 1 or len(n) > 1 or len(o) > 1:
            final_list[i] = '/home'
    final_list = set(final_list)
    return final_list

# TESTING
final_list = get_final_list(links)
print(final_list)
print(len(final_list))

{'/home'}
1


In [7]:
#create a dictionary to for us to store our relevant texts into into 
#their respective categories
school = {'mission' : [], 'curriculum': [], 'philosophy' : [], 'history' : [], 'target' : [], 'resources' : [], 'organizational_factors' : []}
#key words for searching
k = [str(x) for x in np.arange(1900,2012)]
idk = ['advantage']
mission = ['mission',' vision ', 'our purpose', 'cause', 'objectives', 'vision:', 'mission:']
curriculum = ['curriculum', 'program', 'methods', 'pedagogy', 'approach', 'model',  'academics', 'degree']
philosophy = ['philosophy','beliefs', 'principles', 'creed', 'values',  'moral']
history = ['history', 'story', 'background', 'founded', 'established'] + k
target = ['target','gifted', 'at-risk', 'ethnic background', 'target population', 'target audience']
resources = ['resources', 'ESL', 'tutoring', 'after-school programs', 'available resources', 'services', 'opportunities', 'opportunity']
orgfactor = ['organizational_factors', 'statistics', 'API', 'environment', 'ratio', 'average', 'female', 'fund', 'community'  ]
keywords = [(mission[0], mission), (curriculum[0], curriculum), (philosophy[0], philosophy), (history[0], history), ( target[0],target) , (resources[0], resources), (orgfactor[0], orgfactor) ]

#makes all the text in uppercase
for i in range(0,len(keywords)):
    keywords[i] = (keywords[i][0], [s.upper() for s in keywords[i][1]])
keywords[0][1]

['MISSION',
 ' VISION ',
 'OUR PURPOSE',
 'CAUSE',
 'OBJECTIVES',
 'VISION:',
 'MISSION:']

In [8]:
#create a mapper function to map the text which we parse through into 
#its relevant category in the dictionary
# credits: parsing team
def mapper(txt, school):
    
#     print(txt.upper())
    text = txt.upper()
    for i in range(len(keywords)):
        update_with_keyword(keywords[i], txt, school)
        
def update_with_keyword(keyword, txt, school):
    """This functions updates the school dict with txt
    keyword = field name, st mission, history, target, etc"""
    text1 = txt.upper()
#     print(keyword[1])
    for i in range(len(keyword[1])):
        if not isinstance(text1, list):
            text1 = text1.split(keyword[1][i])
        elif len(text1) == 1:
            text1 = text1[0].split(keyword[1][i])
        if len(text1) > 1:
#             print(keyword[0])
            school[keyword[0]].append(txt)
            break

In [9]:
def update_school_dict(final_list, school):
    for link in final_list:
        q = requests.get(url + link)
        k = q.text
        s = BeautifulSoup(k, "lxml")
        j = s.find_all('p')
        for lin in j:
            k = str(lin)
            if lin.get('class') == None and lin.get('a') == None and len(k.split('|')) == 1:
                mapper(' ' + lin.get_text(), school)
    # remove duplicates values with set
    updated_school = dict((k, set(v)) for k, v in school.items())
    return updated_school

In [10]:
updated_school = update_school_dict(final_list, school)
print(updated_school)

{'mission': set(), 'curriculum': set(), 'philosophy': set(), 'history': set(), 'target': set(), 'resources': set(), 'organizational_factors': {' Polk’s Public Safety students put themselves in harm’s way to protect our community.'}}


In [11]:
print(updated_school['mission'])

set()


In [12]:
def save_json(file, school):
    with open(file, 'wb') as outfile:
#         json.dump(school, outfile)
        pickle.dump(school, outfile)


In [13]:
count = 0
def run_master():
    
    #get the links, count of valid website links
    links, count = readtxt(valid)
    result = []
    #runing on first 3 sites
#     to_run = links[0:300]
    for url in links:
        
        
        name, link = url[0], url[1]
        print(link)
#         link = 'https://www.polk.edu/lakeland-gateway-to-college-high-school/'
        links, paragraphs = parse_data(link)
        final_list = get_final_list(links)
        #create a dictionary to for us to store our relevant texts into into 
        #their respective categories
        school = {'school_name': [], 'mission' : [], 'curriculum': [], 'philosophy' : [], 'history' : [], 'target' : [], 'resources' : [], 'organizational_factors' : []}
        try:
            updated_school = update_school_dict(final_list, school)
            updated_school['school_name'] = name
#             print(updated_school)
            print(str(count)+name)
            
            result.append(updated_school)
        except:
            pass
        
        count +=1
#         result[name] =updated_school
    save_json(file, result)    

In [14]:
run_master()

CHARTAUTHN2


MissingSchema: Invalid URL 'CHARTAUTHN2': No schema supplied. Perhaps you meant http://CHARTAUTHN2?